In [1]:
#加载所需库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential  #Sequential 用于初始化神经网络
from keras.layers import Dense       #Dense 用于添加全连接的神经网络层
from keras.layers import LSTM        #LSTM 用于添加长短期内存层
from keras.layers import Dropout     #Dropout 用于添加防止过拟合的dropout层

In [ ]:
#构建训练数据
def train_data():
    dataset_train = pd.read_excel("C:/Users/Administrator/Desktop/Python Process/AA.xlsx")
    training_set = dataset_train.iloc[:, 1:6].values
    dataset_train.head()
        
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    
    X_train = []
    y_train = []
    for i in range(1, 38):
        X_train.append(training_set_scaled[i-1:i, :])
        y_train.append(training_set_scaled[i, :])

    X_train, y_train = np.array(X_train), np.array(y_train)      
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 6))
    
    return X_train,y_train,dataset_train,sc
#构建测试数据
def test_data(dataset_train,sc):
    dataset_test = pd.read_csv('tatatest.csv')
    real_stock_price = dataset_test.iloc[:, 1:2].values
    #合并训练集和测试集
    dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
    db_all = dataset_total.iloc[:, 1:7].values

    inputs = db_all[len(dataset_total) - len(dataset_test) - 60:]
    inputs = inputs.reshape(-1,6)

    inputs = sc.transform(inputs)
    X_test = []
    for i in range(60, 76):
        X_test.append(inputs[i-60:i, :])
        #X_test.append(inputs[i-60:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 6))
    
    return X_test,real_stock_price
#创建股票预测模型
def stock_model(X_train, y_train):
    regressor = Sequential()
    #LSTM的输入为 [samples, timesteps, features],这里的timesteps为步数，features为维度 这里我们的数据是6维的
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 6)))
    
    regressor.add(Dropout(0.2))
    
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    #全连接，输出6个
    regressor.add(Dense(units = 6))
    
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
    
    return regressor

def main():
    X_train, y_train,dataset_train,sc = train_data()
    
    regressor = stock_model(X_train, y_train)
    X_test,real_stock_price = test_data(dataset_train,sc)
    predicted_stock_price = regressor.predict(X_test)
    
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    
    plt.plot(real_stock_price, color = 'black', label = 'TATA Stock Price')
    #显示开盘价
    plt.plot(predicted_stock_price[:,0], color = 'green', label = 'Predicted TATA Stock Price')
    plt.title('TATA Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('TATA Stock Price')
    plt.legend()
    plt.show()

if __name__ == '__main__':
    main()
